In [1]:
#Instalo pylinac e importo todo lo que necesito
!pip install pylinac
import datetime
import pydicom
import os
from pylinac.picketfence import PicketFence, MLC, Orientation
from google.colab import drive
#Conecto con mi drive y genero la ruta a mi carpeta de picket fence (PF- 1)
drive.mount('/content/drive')
my_directory = '/content/drive/My Drive/PPS - Intecnus/Python/PF- 1'
os.makedirs(my_directory, exist_ok=True)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.9/840.9 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 382.9/382.9 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 13.8 MB/s eta 0:00:00
  Created wheel for future: filename=future-0.18.3-py3-none-any.whl size=492025 sha256=9c7568fbc78754ac65a8bb9cd6e6cf83efe1e07959dcfa6eba5eea4196712288
  Stored in directory: /root/.cache/pip/wheels/da/19/ca/9d8c44cd311a955509d7e13da3f0bea42400c469ef825b580b
Successfully built future
  Attempting uninstall: future
    Found existing installation: future 1.0.0
    Uninstalling future-1.0.0:
      Successfully uninstalled future-1.0.0
Mounted at /content/drive


In [2]:
# Muestro los archivos en mi carpeta para elegir uno.
# Si no se analizo la imagen antes, la misma viene sin extensión, en ese caso se la
# agregamos. Si se elige un archivo que no es una imagen DICOM o el número que se eligio no está en la lista
# se vuelven a mostrar todos los archivos para que se vuelva a elegir. También guardo el nombre del archivo
# de mi imagen para usarlo en el nombre del pdf
def seleccionar_archivo_y_verificar_extension(directorio):
    archivos = os.listdir(directorio)
    if not archivos:
        print("No hay archivos en el directorio.")
        return None, None

    archivo_seleccionado = None
    while archivo_seleccionado is None:
        print("\nArchivos disponibles:")
        for i, archivo in enumerate(archivos):
            print(f"{i}: {archivo}")

        try:
            indice = int(input("\nSelecciona el número del archivo que deseas: "))
            if 0 <= indice < len(archivos):
                archivo_completo = archivos[indice]
                nombre_archivo, extension = os.path.splitext(archivo_completo)

                if extension == '.dcm':
                    ruta_final = os.path.join(directorio, archivo_completo)
                    print(f"El archivo ya tiene la extensión .dcm: {ruta_final}")

                elif extension != '':
                    print("El archivo seleccionado no se corresponde con una imagen DICOM.")
                    archivo_seleccionado = None
                    continue

                else:
                    ruta_original = os.path.join(directorio, archivo_completo)
                    ruta_final = os.path.join(directorio, archivo_completo + '.dcm')
                    os.rename(ruta_original, ruta_final)
                    print(f"Archivo renombrado a: {ruta_final}")

                return ruta_final, nombre_archivo
            else:
                print("El número seleccionado no es válido. Por favor, selecciona uno de los números mostrados.")
        except ValueError:
            print("Por favor, ingresa un número válido.")

# Utilizo los metadatos de mi imagen DICOM para saber en que fech y hora fue realizada la imagen.
# Los guardo para ponerlos en el pdf.
def Metadata(pf_img):
    dicom_dataset = pydicom.dcmread(pf_img)
    fecha_cruda = dicom_dataset.StudyDate
    fecha = fecha_cruda[:4] + "." + fecha_cruda[4:6] + "." + fecha_cruda[6:]
    hora_cruda = dicom_dataset.StudyTime
    hora = hora_cruda[:2] + ":" + hora_cruda[2:4]
    return fecha, hora

def main():
    """Función principal del programa."""
    ruta_final, nombre_archivo = seleccionar_archivo_y_verificar_extension(my_directory)
    print(f"Proceso finalizado. Archivo seleccionado: {ruta_final}")
    pf_img = ruta_final
    pf1 = PicketFence(pf_img, mlc=MLC.AGILITY) #En mlc elegimos el modelo del MLC, si no estuviera en pylinac se puede configurar de forma manual
    #el height_threshold lo disminuyo para disminuir el umbral que diferencia el fondo del picket, la idea de disminuirlo es para que detecte más hojas
    #el edge_threshold lo aumento para aumentar la desviación estándar que puede haber
    #también nos permite agarrar o quitar láminas de los bordes
    pf1.analyze(tolerance=0.5, action_tolerance=0.3, height_threshold = 0.5, edge_threshold= 3)
    current_datetime = datetime.datetime.now()
    formatted_datetime = current_datetime.strftime("%Y-%m-%d")
    fecha, hora = Metadata(pf_img)
    metadata = {"Fecha de la imagen": fecha + "; " + hora}
    logo_path = os.path.join(my_directory, 'logo.png')
    pdf_path = os.path.join(my_directory, f"Picket Fence {nombre_archivo} _ {formatted_datetime}.pdf")
    pf1.publish_pdf(filename=pdf_path, open_file=True, logo=logo_path, metadata=metadata)
if __name__ == "__main__":
    main()


Archivos disponibles:
0: PFG0.dcm
1: PFG0VDR.dcm
2: PFG90.dcm
3: PFG270.dcm
4: logo.png
5: imagen5.dcm
6: imagen3.dcm
7: imagen1.dcm
8: imagen4.dcm
9: PF - Imagen 5.ipynb
10: Picket Fence Imagen 1 2025-01-03.pdf
11: imagen2.dcm
12: PF - Imagen 1.ipynb
13: PF - Imagen 3.ipynb
14: PF - Imagen 2.ipynb
15: Picket Fence Imagen 1 2025-01-27.pdf
16: 254106104153711.dcm
17: 25430610437865.dcm
18: Picket Fence imagen5 2025-01-28.pdf
19: Picket Fence imagen3 2025-01-28.pdf
20: Picket Fence imagen1 2025-01-29.pdf
21: Picket Fence PFG270 2025-01-29.pdf
22: Picket Fence imagen5 2025-01-29.pdf
23: PF - Imagen 4.ipynb
24: Picket Fence 254106104153711 2025-01-29.pdf
25: Picket Fence 25430610437865 2025-01-29.pdf
26: Picket Fence imagen1 2025-01-30.pdf
27: Picket Fence imagen5 2025-02-11.pdf
28: Picket Fence imagen3 2025-02-11.pdf
29: Picket Fence imagen1 2025-02-11.pdf
30: Picket Fence imagen2 2025-02-11.pdf
31: Picket Fence 254106104153711 2025-02-11.pdf
32: Picket Fence 25430610437865 2025-02-11.pd

<frozen importlib._bootstrap>:1047: ImportWarning: _PyDrive2ImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _GenerativeAIImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _AltairImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _PyDrive2ImportHook.find_spec() not found; falling back to find_module()
<fro